# How to get MOC data from VESPA in Python

Illustrate search for CDPP TAP MOC for vespa using 'pyvo' and 'astropy'

## Import required modules

In [65]:
import pyvo as vo
from mpl_toolkits.basemap import Basemap
import re

## Search for a specific TAP service

We are looking for VVEx - VIRTIS/Venus Express database since it has a MOC

In [66]:
tap_services = vo.registry.search(servicetype='tap',keywords=['VVEx'], includeaux=True)
print(tap_services.to_table)

<bound method DALResults.to_table of <Table length=1>
           ivoid                  res_type     ...  intf_types  intf_roles
                                               ...                        
           object                  object      ...    object      object  
---------------------------- ----------------- ... ------------ ----------
ivo://padc.obspm.planeto/tap vs:catalogservice ... vs:paramhttp        std>


## Get the first result and print its data

In [67]:
resource = tap_services[0]
print ("\nLength: ",len(resource))
print ("\nRessource: ",resource)
print("\nUrl: ",resource.access_url)
print("\nTables object: ",resource.service.tables)


Length:  17

Ressource:  ('ivo://padc.obspm.planeto/tap', 'vs:catalogservice', 'PADCC TAP', 'PADC TAP Server on voparis-tap-planeto.obspm.fr TAP service', '', "The PADC TAP Server on voparis-tap-planeto.obspm.fr's TAP end point. The Table Access\nProtocol (TAP) lets you execute queries against our database tables,\ninspect various metadata, and upload your own data. It is thus the\nVO's premier way to access public data holdings.\n\nTables exposed through this endpoint include: epn_core from the apis schema, epn_core from the basecom schema, epn_core, observatories from the bdip schema, epn_core from the dynastvo schema, epn_core from the exoplanet schema, epn_core from the hst_planeto schema, epn_core from the iks schema, epn_core from the m4ast schema, epn_core from the meteor_showers schema, epn_core from the pds_speclib schema, epn_core from the planets schema, epn_core from the spectro_asteroids schema, epn_core from the spectro_planets schema, columns, groups, key_columns, keys,

## Get the 1st table associated to it

In [78]:
resourceKey = list(resource.get_tables(table_limit=30).keys())[0]
mytable = resource.get_tables(table_limit=30)[resourceKey]

## Check if a a table has a coverage key (and so maybe a MOC)

In [84]:
def doContainMOC(table) :
    for field in table.columns :
        if re.search("^.*<BaseParam name=\"coverage\"/>.*$", str(field)) :
            return True
    return False
        
print(doContainMOC(mytable))

True


In [87]:
# for field in mytable.columns :
#        print(field)

<BaseParam name="granule_uid"/>
<BaseParam name="granule_gid"/>
<BaseParam name="obs_id"/>
<BaseParam name="dataproduct_type"/>
<BaseParam name="target_name"/>
<BaseParam name="target_class"/>
<BaseParam name="time_min"/>
<BaseParam name="time_max"/>
<BaseParam name="time_sampling_step_min"/>
<BaseParam name="time_sampling_step_max"/>
<BaseParam name="time_exp_min"/>
<BaseParam name="time_exp_max"/>
<BaseParam name="spectral_range_min"/>
<BaseParam name="spectral_range_max"/>
<BaseParam name="spectral_sampling_step_min"/>
<BaseParam name="spectral_sampling_step_max"/>
<BaseParam name="spectral_resolution_min"/>
<BaseParam name="spectral_resolution_max"/>
<BaseParam name="c1min"/>
<BaseParam name="c1max"/>
<BaseParam name="c2min"/>
<BaseParam name="c2max"/>
<BaseParam name="c3min"/>
<BaseParam name="c3max"/>
<BaseParam name="s_region"/>
<BaseParam name="c1_resol_min"/>
<BaseParam name="c1_resol_max"/>
<BaseParam name="c2_resol_min"/>
<BaseParam name="c2_resol_max"/>
<BaseParam name="c3_